Southern University of Science and Technology-Department of Computer Science and Engineering

Course: Machine Learning(CS 405)-Professor: Qi Hao

## Homework #2
#### Due date: October, 7th, 2019

In [1]:
"""
Import libraries that you might require.

"""

import math
import operator

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

We will implement the KNN algorithm for the breast cancer dataset. Refer to the pdf and the following functions for the instructions. Complete all the functions as indicated below. The four functions would be autograded as mentioned in the pdf.

In [75]:
"""
Task 1: Classification

Please implement KNN for K: 3, 5, and 7 with the following norms:
L1
L2
L-inf
"""

# Read data (Breast Cancer Dataset). Remember to comment out the code not contained in a function.
from math import sqrt

from sklearn.datasets import load_breast_cancer

breast = load_breast_cancer()

X = breast['data']
y = breast['target']

np.random.seed(100)
p = np.random.permutation(len(X))
X, y = X[p], y[p]

X_train, y_train = X[:400], y[:400]
X_val, y_val = X[400:500], y[400:500]
X_test, y_test = X[500:], y[500:]


def distanceFunc(metric_type, vec1, vec2):
    """
    Computes the distance between two d-dimension vectors. 
    
    Please DO NOT use Numpy's norm function when implementing this function. 
    
    Args:
        metric_type (str): Metric: L1, L2, or L-inf
        vec1 ((d,) np.ndarray): d-dim vector
        vec2 ((d,)) np.ndarray): d-dim vector
    
    Returns:
        distance (float): distance between the two vectors
    """

    diff = vec1 - vec2
    if metric_type == "L1":
        distance = np.sum(np.abs(diff))

    if metric_type == "L2":
        distance = sqrt(np.sum(np.square(diff)))
        
    if metric_type == "L-inf":
        distance = max(np.abs(diff))
        
    return distance


def computeDistancesNeighbors(K, metric_type, X_train:np.ndarray, y_train, sample):
    """
    Compute the distances between every datapoint in the train_data and the 
    given sample. Then, find the k-nearest neighbors.
    
    Return a numpy array of the label of the k-nearest neighbors.
    
    Args:
        K (int): K-value
        metric_type (str): metric type
        X_train ((n,p) np.ndarray): Training data with n samples and p features
        y_train : Training labels
        sample ((p,) np.ndarray): Single sample whose distance is to computed with every entry in the dataset
        
    Returns:
        neighbors (list): K-nearest neighbors' labels
    """

    # You will also call the function "distanceFunc" here
    # Complete this function
    (m, n) = X_train.shape
    p = sample.shape[0]
    
    dist = np.zeros((p, m))
    
    for i in range(0, p):
        for j in range(0, m):
            dist[i][j] = distanceFunc(metric_type, sample[i], X_train[j])
            
        
    indices = np.argsort(dist, kind='stable')
    lst = []
    
    for i in range(p):
        lst.append(y_train[indices[i, :K]])
            
            
    return np.array(lst)


def Majority(neighbors):
    """
    Performs majority voting and returns the predicted value for the test sample.
    
    Since we're performing binary classification the possible values are [0,1].
    
    Args:
        neighbors (list): K-nearest neighbors' labels
        
    Returns:
        predicted_value (int): predicted label for the given sample
    """
    
    # Performs majority voting
    # Complete this function
    res = np.argmax(np.bincount(neighbors))
    
    return res


def KNN(K, metric_type, X_train, y_train, X_val:np.ndarray):
    """
    Returns the predicted values for the entire validation or test set.
    
    Please DO NOT use Scikit's KNN model when implementing this function. 

    Args:
        K (int): K-value
        metric_type (str): metric type
        X_train ((n,p) np.ndarray): Training data with n samples and p features
        y_train : Training labels
        X_val ((n, p) np.ndarray): Validation or test data
        
    Returns:
        predicted_values (list): output for every entry in validation/test dataset 
    """
    
    # Complete this function
    # Loop through the val_data or the test_data (as required)
    # and compute the output for every entry in that dataset  
    # You will also call the function "Majority" here
    K_nearest = computeDistancesNeighbors(K, metric_type, X_train, y_train, X_val)

    predictions = np.zeros((X_val.shape[0],), dtype='int64')

    for i in range(X_val.shape[0]):
        predictions[i] = Majority(K_nearest[i])
        

    return predictions


def evaluation(predicted_values, actual_values):
    """
    Computes the accuracy of the given datapoints.
    
    Args:
        predicted_values ((n,) np.ndarray): Predicted values for n samples
        actual_values ((n,) np.ndarray): Actual values for n samples
    
    Returns:
        accuracy (float): accuracy
    """
    
    return accuracy_score(predicted_values, actual_values)


def main():
    """
    Calls the above functions in order to implement the KNN algorithm.
    
    Test over the following range K = 3,5,7 and all three metrics. 
    In total you will have nine combinations to try.
    
    PRINTS out the accuracies for the nine combinations on the validation set,
    and the accuracy on the test set for the selected K value and appropriate norm.
    
    REMEMBER: You have to report these values by populating the Table 1.
    """
    
    ## Complete this function
    
    K = [3,5,7]
    norm = ["L1", "L2", "L-inf"]
    
    print("<<<<VALIDATION DATA PREDICTIONS>>>>")
    print(32*"-")
    print("|{:^6s}|{:^6s}|{:^16s}|".format("K", "Norm", "Accuracy(%)"))

    for k in K:
        for n in norm:
            res = KNN(k, n, X_train, y_train, X_val)
            acc = evaluation(y_val, res)
            print("|{:^6d}|{:^6s}|{:^16%}|".format(k, n, acc))
            
        print(32*"-")


    
    ## Complete
    print("<<<<TEST DATA PREDICTIONS>>>>")
    print(32*"-")
    print("|{:^6s}|{:^6s}|{:^16s}|".format("K", "Norm", "Accuracy(%)"))

    for k in K:
        for n in norm:
            res = KNN(k, n, X_train, y_train, X_test)
            acc = evaluation(y_test, res)
            print("|{:^6d}|{:^6s}|{:^16%}|".format(k, n, acc))

        print(32*"-")

    ## Complete

Uncomment the code below to run the main function (Remember to recomment the code before submitting).

In [76]:
# Finally, call the main function
main()

<<<<VALIDATION DATA PREDICTIONS>>>>
--------------------------------
|  K   | Norm |  Accuracy(%)   |
|  3   |  L1  |   94.000000%   |
|  3   |  L2  |   95.000000%   |
|  3   |L-inf |   94.000000%   |
--------------------------------
|  5   |  L1  |   94.000000%   |
|  5   |  L2  |   93.000000%   |
|  5   |L-inf |   94.000000%   |
--------------------------------
|  7   |  L1  |   93.000000%   |
|  7   |  L2  |   92.000000%   |
|  7   |L-inf |   93.000000%   |
--------------------------------
<<<<TEST DATA PREDICTIONS>>>>
--------------------------------
|  K   | Norm |  Accuracy(%)   |
|  3   |  L1  |   88.405797%   |
|  3   |  L2  |   88.405797%   |
|  3   |L-inf |   89.855072%   |
--------------------------------
|  5   |  L1  |   91.304348%   |
|  5   |  L2  |   89.855072%   |
|  5   |L-inf |   89.855072%   |
--------------------------------
|  7   |  L1  |   89.855072%   |
|  7   |  L2  |   91.304348%   |
|  7   |L-inf |   89.855072%   |
--------------------------------


Answer the following questions here:

1. How could having a larger dataset influence the performance of KNN?

2. Tabulate your results from `main()` in the table provided.

3. Finally, mention the best K and the norm combination you have settled upon and report the accuracy on the test set using that combination.

1. KNN 对于大数据量的样本比哦阿羡比较差，因为中间主要步骤涉及到两两比较，如不优化会有 $O(n^2)$ 级别的复杂度.
2. 
```
<<<<VALIDATION DATA PREDICTIONS>>>>
--------------------------------
|  K   | Norm |  Accuracy(%)   |
|  3   |  L1  |   94.000000%   |
|  3   |  L2  |   95.000000%   |
|  3   |L-inf |   94.000000%   |
--------------------------------
|  5   |  L1  |   94.000000%   |
|  5   |  L2  |   93.000000%   |
|  5   |L-inf |   94.000000%   |
--------------------------------
|  7   |  L1  |   93.000000%   |
|  7   |  L2  |   92.000000%   |
|  7   |L-inf |   93.000000%   |
--------------------------------
<<<<TEST DATA PREDICTIONS>>>>
--------------------------------
|  K   | Norm |  Accuracy(%)   |
|  3   |  L1  |   88.405797%   |
|  3   |  L2  |   88.405797%   |
|  3   |L-inf |   89.855072%   |
--------------------------------
|  5   |  L1  |   91.304348%   |
|  5   |  L2  |   89.855072%   |
|  5   |L-inf |   89.855072%   |
...
|  7   |  L1  |   89.855072%   |
|  7   |  L2  |   91.304348%   |
|  7   |L-inf |   89.855072%   |
--------------------------------
```
3. 对于 X_val, 取 K = 3, Norm = L2; 对于 X_test, 取 K = 5, Norm = L1. (K = 7, Norm = L1 也有相同的结果，但是其计算复杂度大于 K = 5).